# What am I Investigating?
Tennis is a racquet sport that can be played individually against a single opponent (singles) or between two teams of two players each (doubles). Players use a strung racquet to strike a hollow rubber ball, covered with felt, over a net and into the opponent's court. The objective of the game is to play the ball in such a way that the opponent cannot play a valid return. The player who is unable to return the ball will not gain a point, while the opposite player will.  
Tennis is played on different surfaces, including grass, clay, hard courts, and even indoor carpet. Each surface affects the ball's speed and bounce differently, leading to varied styles of play. In this project I'm investigating the relationship between the type of surface on which the match was played on to various stats (such as the duration of the match, the number of aces, etc.).  


---


The data is taken from kaggle and is called ['Huge Tennis Database'](https://www.kaggle.com/datasets/guillemservera/tennis). This dataset is a comprehensive collection of ATP tennis rankings, match results, and player statistics. It is derived from the original database created and maintained by Jeff Sackmann, which could be found in the [following github repository](https://github.com/JeffSackmann/tennis_atp).  
On kaggle there are two methods for importing this data - by using SQLlite or by using CSV files. I chose to download the CSV files regarding all the ATP matches conducted between 1968 and 2023. Because the data of each year's matches is stored in a separate CSV file, I had to concatenate them.

# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import skew, kurtosis

pd.set_option('display.max_columns', 50)

# Loading Data

In [2]:
folder_path = '/content/drive/MyDrive/Tennis_Analysis/full_matches_data'

# List all files in the directory with a .csv extension
all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.csv')]

# Use a list comprehension to read each file into a dataframe and then concatenate them all
combined_df = pd.concat([pd.read_csv(os.path.join(folder_path, f)) for f in all_files], ignore_index=True)

In [3]:
combined_df.sample(10)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
84178,2016-M004,Acapulco,Hard,32,A,20160222,281,105992,NaN,Q,Ryan Harrison,R,183.0,USA,23.7,105227,3.0,NaN,Marin Cilic,R,198.0,CRO,27.4,6-3 3-6 6-2,3,R32,105.0,6.0,3.0,66.0,36.0,30.0,18.0,13.0,2.0,4.0,5.0,4.0,75.0,44.0,34.0,10.0,13.0,1.0,5.0,127.0,471.0,12.0,2555.0
34497,2000-407,Rotterdam,Hard,32,A,20000214,15,102562,NaN,NaN,Jiri Novak,R,190.0,CZE,24.9,102563,NaN,NaN,Thomas Johansson,R,180.0,SWE,24.8,4-6 6-3 6-2,3,R32,112.0,6.0,1.0,82.0,58.0,45.0,10.0,14.0,3.0,5.0,12.0,9.0,105.0,54.0,44.0,16.0,13.0,12.0,16.0,40.0,859.0,41.0,847.0
45311,2003-499,Delray Beach,Hard,32,A,20030303,10,103581,NaN,NaN,Flavio Saretta,R,183.0,BRA,22.6,102834,NaN,Q,Paul Goldstein,R,178.0,USA,26.5,6-4 6-4,3,R32,94.0,3.0,1.0,67.0,43.0,30.0,13.0,10.0,5.0,6.0,1.0,3.0,68.0,36.0,24.0,15.0,10.0,4.0,7.0,88.0,450.0,202.0,166.0
13177,1994-615,Dusseldorf,Clay,32,A,19940516,2,101843,NaN,NaN,Sergi Bruguera,R,188.0,ESP,23.3,101434,NaN,NaN,Petr Korda,L,190.0,CZE,26.3,6-0 4-6 6-1,3,RR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2579.0,12.0,1951.0
51145,2005-416,Rome Masters,Clay,64,M,20050502,31,104597,NaN,Q,Nicolas Almagro,R,183.0,ESP,19.6,104214,NaN,NaN,Igor Andreev,R,185.0,RUS,21.8,3-6 6-4 6-1,3,R64,112.0,2.0,2.0,97.0,58.0,38.0,22.0,13.0,8.0,11.0,2.0,3.0,72.0,45.0,28.0,14.0,13.0,3.0,8.0,101.0,419.0,37.0,954.0
12550,1994-425,Barcelona,Clay,56,A,19940404,19,102148,NaN,NaN,Fabrice Santoro,R,178.0,FRA,21.3,102184,NaN,Q,David Prinosil,R,185.0,GER,21.0,6-2 6-4,3,R64,75.0,2.0,2.0,51.0,37.0,29.0,10.0,9.0,3.0,3.0,4.0,1.0,54.0,29.0,20.0,12.0,9.0,4.0,7.0,111.0,406.0,120.0,378.0
51914,2005-505,Vina del Mar,Clay,32,A,20050131,28,103602,2.0,NaN,Fernando Gonzalez,R,183.0,CHI,24.5,103151,5.0,NaN,Mariano Zabaleta,R,183.0,ARG,26.9,4-6 7-6(4) 7-6(2),3,QF,176.0,9.0,6.0,122.0,65.0,46.0,35.0,17.0,6.0,9.0,6.0,7.0,126.0,82.0,57.0,20.0,17.0,10.0,12.0,14.0,1405.0,49.0,770.0
46604,2004-1536,Madrid Masters,Hard,48,M,20041018,20,104026,7.0,NaN,Joachim Johansson,R,198.0,SWE,22.2,103084,NaN,NaN,Guillermo Canas,R,185.0,ARG,26.8,7-6(3) 7-5,3,R32,112.0,23.0,5.0,62.0,36.0,35.0,17.0,12.0,0.0,0.0,3.0,3.0,83.0,48.0,37.0,17.0,12.0,6.0,7.0,11.0,1505.0,16.0,1369.0
61735,2008-540,Wimbledon,Grass,128,G,20080623,83,104198,NaN,NaN,Guillermo Garcia Lopez,R,188.0,ESP,25.0,104597,12.0,NaN,Nicolas Almagro,R,183.0,ESP,22.8,6-3 3-6 5-7 6-1 6-2,5,R64,161.0,12.0,9.0,131.0,83.0,68.0,19.0,23.0,3.0,7.0,13.0,6.0,125.0,64.0,45.0,31.0,22.0,8.0,15.0,104.0,437.0,12.0,1485.0
37410,2001-338,Sydney,Hard,32,A,20010108,19,102035,NaN,NaN,Jonas Bjorkman,R,183.0,SWE,28.7,101727,NaN,NaN,Jason Stoltenberg,R,185.0,AUS,30.7,6-4 3-6 7-6(3),3,R16,134.0,1.0,3.0,102.0,59.0,40.0,23.0,16.0,7.0,11.0,5.0,4.0,103.0,60.0,45.0,19.0,15.0,5.0,8.0,44.0,845.0,63.0,641.0


# Understanding the Big Picture

Getting the number of rows and columns in the dataset (pre-cleaning):

In [4]:
number_of_rows_pre_cleaning = combined_df.shape[0]
number_of_columns_pre_cleaning = combined_df.shape[1]

print(f"The dataset has {number_of_rows_pre_cleaning} rows and {number_of_columns_pre_cleaning} columns.")

The dataset has 104682 rows and 49 columns.


Examining the data type of all the columns in the dataset:

In [5]:
combined_df.dtypes

tourney_id             object
tourney_name           object
surface                object
draw_size               int64
tourney_level          object
tourney_date            int64
match_num               int64
winner_id               int64
winner_seed           float64
winner_entry           object
winner_name            object
winner_hand            object
winner_ht             float64
winner_ioc             object
winner_age            float64
loser_id                int64
loser_seed            float64
loser_entry            object
loser_name             object
loser_hand             object
loser_ht              float64
loser_ioc              object
loser_age             float64
score                  object
best_of                 int64
round                  object
minutes               float64
w_ace                 float64
w_df                  float64
w_svpt                float64
w_1stIn               float64
w_1stWon              float64
w_2ndWon              float64
w_SvGms   

*After examining the data types of all the columns, I can see a problem with some of the columns. Though some of the variables are discrete, Pandas falsely attributed them to `float64`.  
In the data cleaning section I'll change the data type of those columns to the smaller data type in order to reduce memory usage.*

Getting the descriptive statistics of the numeric variables:

In [6]:
combined_df.describe()

,draw_size,tourney_date,match_num,winner_id,winner_seed,winner_ht,winner_age,loser_id,loser_seed,loser_ht,loser_age,best_of,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
count,104682.000000,1.046820e+05,104682.000000,104682.000000,42400.000000,102228.000000,104677.000000,104682.000000,23300.000000,99827.000000,104664.000000,104682.000000,91646.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94476.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94475.000000,94476.000000,94475.000000,94475.000000,103493.000000,102505.000000,102146.000000,101163.000000
mean,53.518274,2.005813e+07,72.470014,106703.119075,6.923821,185.717123,25.770551,106813.711555,8.291545,185.210985,25.884222,3.441394,103.830893,6.526256,2.733728,78.128976,47.657888,35.933390,16.729124,12.409395,3.526488,5.164499,4.840857,3.484594,80.967071,48.089749,31.955067,14.983604,12.209736,4.813242,8.739995,80.657300,1429.137193,119.051084,895.639414
std,38.654709,9.444893e+04,113.061314,15824.619341,6.225644,6.661203,3.807532,16002.830220,6.735971,6.610041,3.931021,0.829437,39.529188,5.337348,2.364958,29.539587,19.242839,13.852961,6.983693,4.128099,3.086376,4.062636,4.680657,2.620083,29.471466,19.402443,14.462356,7.215126,4.136927,3.276748,4.134577,134.428663,1753.613904,179.584694,1002.257276
min,2.000000,1.990123e+07,1.000000,100284.000000,1.000000,160.000000,14.300000,100282.000000,1.000000,160.000000,14.500000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,32.000000,1.997101e+07,10.000000,102148.000000,3.000000,180.000000,23.000000,102154.000000,4.000000,180.000000,23.000000,3.000000,75.000000,3.000000,1.000000,56.000000,34.000000,26.000000,12.000000,9.000000,1.000000,2.000000,2.000000,2.000000,59.000000,34.000000,22.000000,10.000000,9.000000,2.000000,6.000000,18.000000,529.000000,37.000000,395.000000
50%,32.000000,2.005082e+07,24.000000,103602.000000,5.000000,185.000000,25.500000,103566.000000,6.000000,185.000000,25.700000,3.000000,96.000000,5.000000,2.000000,73.000000,44.000000,33.000000,16.000000,11.000000,3.000000,4.000000,4.000000,3.000000,76.000000,45.000000,30.000000,14.000000,11.000000,4.000000,8.000000,46.000000,880.000000,70.000000,658.000000
75%,64.000000,2.014022e+07,73.000000,104797.000000,8.000000,190.000000,28.300000,104919.000000,11.000000,190.000000,28.500000,3.000000,125.000000,9.000000,4.000000,94.000000,58.000000,43.000000,21.000000,15.000000,5.000000,7.000000,7.000000,5.000000,97.000000,58.000000,40.000000,19.000000,15.000000,7.000000,11.000000,89.000000,1598.000000,119.000000,1040.000000
max,128.000000,2.023083e+07,1701.000000,211468.000000,35.000000,211.000000,42.700000,212041.000000,35.000000,211.000000,46.000000,5.000000,1146.000000,113.000000,26.000000,491.000000,361.000000,292.000000,82.000000,90.000000,24.000000,34.000000,103.000000,26.000000,489.000000,328.000000,284.000000,101.000000,91.000000,28.000000,38.000000,2101.000000,16950.000000,2159.000000,16950.000000


Counting the number of NA's in each column:

In [ ]:
combined_df.isna().sum()

*While examining the number of NA's in each column, my main focus is on the number of NA's in the variables depicting the match stats, because I intened the analyze them.*

Checking if there are any duplicate rows:

In [8]:
number_of_duplicated_rows = combined_df.duplicated().sum()
print(f"There are {number_of_duplicated_rows} duplicated rows in the dataset.")

There are 0 duplicated rows in the dataset.


# Data Cleaning

Changing the format of the dates in the tourney_date column, from `YYYYMMDD` format to `Y M D` format.

In [9]:
combined_df["tourney_date"] = pd.to_datetime(combined_df['tourney_date'], format='%Y%m%d')
combined_df["tourney_date"].head()

0   1990-12-31
1   1990-12-31
2   1990-12-31
3   1990-12-31
4   1990-12-31
Name: tourney_date, dtype: datetime64[ns]

Dividing the 'tourney_date' column to year, month and day

In [10]:
combined_df["Year"] = combined_df["tourney_date"].dt.strftime('%Y')
combined_df["Month"] = combined_df["tourney_date"].dt.strftime('%m')
combined_df["Day"] = combined_df["tourney_date"].dt.strftime('%d')
combined_df[["Year", "Month", "Day"]].head(10)

,Year,Month,Day
0,1990,12,31
1,1990,12,31
2,1990,12,31
3,1990,12,31
4,1990,12,31
5,1990,12,31
6,1990,12,31
7,1990,12,31
8,1990,12,31
9,1990,12,31


Deleting matches (rows) that we don't know their statistics:

In [11]:
match_stats_columns_names = ["w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced"]

matches_without_stats_df = combined_df[combined_df[match_stats_columns_names].isna().any(axis=1)]

matches_without_stats_df.head(10)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,...,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,Year,Month,Day
7,1991-339,Adelaide,Hard,32,A,1990-12-31,8,101320,5.0,NaN,Magnus Gustafsson,R,185.0,SWE,23.9,100713,NaN,WC,John Fitzgerald,R,183.0,AUS,30.0,6-2 6-3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,NaN,188.0,NaN,1990,12,31
61,1991-354,Wellington,Hard,32,A,1990-12-31,31,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,101767,NaN,NaN,Lars Jonsson,R,188.0,SWE,20.5,6-1 6-4 6-4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,NaN,100.0,NaN,1990,12,31
87,1991-301,Auckland,Hard,32,A,1991-01-07,26,100954,5.0,NaN,Luiz Mattar,R,183.0,BRA,27.3,101377,NaN,NaN,Christian Bergstrom,R,180.0,SWE,23.4,W/O,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,645.0,69.0,491.0,1991,01,07
88,1991-301,Auckland,Hard,32,A,1991-01-07,27,101120,4.0,NaN,Karel Novacek,R,190.0,CZE,25.7,101767,NaN,NaN,Lars Jonsson,R,188.0,SWE,20.5,W/O,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,759.0,84.0,432.0,1991,01,07
251,1991-D016,Davis Cup G1 QF: BRA vs PER,Hard,4,D,1991-02-01,1,100954,NaN,NaN,Luiz Mattar,R,183.0,BRA,27.4,101074,NaN,NaN,Carlos Di Laura,L,175.0,PER,26.2,6-7(3) 6-3 6-2 6-1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,691.0,NaN,NaN,1991,02,01
252,1991-D016,Davis Cup G1 QF: BRA vs PER,Hard,4,D,1991-02-01,2,101761,NaN,NaN,Jaime Oncins,R,196.0,BRA,20.6,101641,NaN,NaN,Jose Luis Noriega,R,178.0,PER,21.3,6-3 6-2 6-2,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0,345.0,NaN,NaN,1991,02,01
253,1991-D016,Davis Cup G1 QF: BRA vs PER,Hard,4,D,1991-02-01,4,100954,NaN,NaN,Luiz Mattar,R,183.0,BRA,27.4,101641,NaN,NaN,Jose Luis Noriega,R,178.0,PER,21.3,6-2 6-1 7-5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,691.0,NaN,NaN,1991,02,01
254,1991-D036,Davis Cup G1 QF: DEN vs MAR,Carpet,4,D,1991-02-01,1,101775,NaN,NaN,Frederik Fetterlein,R,190.0,DEN,20.5,102223,NaN,NaN,Karim Alami,R,185.0,MAR,17.6,6-4 6-3 6-1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.0,82.0,339.0,50.0,1991,02,01
255,1991-D036,Davis Cup G1 QF: DEN vs MAR,Carpet,4,D,1991-02-01,2,101264,NaN,NaN,Michael Tauson,R,193.0,DEN,24.5,101962,NaN,NaN,Younes El Aynaoui,R,193.0,MAR,19.3,6-1 6-3 6-3,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.0,260.0,348.0,48.0,1991,02,01
256,1991-D036,Davis Cup G1 QF: DEN vs MAR,Carpet,4,D,1991-02-01,4,101264,NaN,NaN,Michael Tauson,R,193.0,DEN,24.5,102223,NaN,NaN,Karim Alami,R,185.0,MAR,17.6,6-3 6-4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132.0,260.0,339.0,50.0,1991,02,01


In [14]:
matches_without_stats_df.groupby(by=["Year", "tourney_level"], as_index=False)[match_stats_columns_names].apply(lambda x: x.isna().sum())

,Year,tourney_level,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced
0,1990,A,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,1991,A,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
2,1991,D,334,334,334,334,334,334,334,334,334,334,334,334,334,334,334,334,334,334
3,1991,F,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1991,G,88,88,88,88,88,88,88,88,88,88,88,88,88,88,88,88,88,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022,M,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
132,2023,A,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
133,2023,D,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96,96
134,2023,G,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [ ]:
combined_df.dropna(subset=match_stats_columns_names, inplace=True)

combined_df[match_stats_columns_names].isna().sum() # If the code above worked, the columns in the list above should have 0 NA's.

In the "understanding the big picture" section I counted the number of [NA's in each column](https://colab.research.google.com/drive/1YiNo6ZBjWNuTTT72mS8qQZ_2W1ZzAFnL#scrollTo=INYVRYc3RsLx&line=1&uniqifier=1). As can be seen, we don't have the stats for 10,207 matches - which is 9.75% of the matches we had in the initial dataframe (we had information on 104,682 at the beginning).

Converting all the match stats columns from float64 to int64:

In [ ]:
for col in match_stats_columns_names:
    combined_df[col] = pd.to_numeric(combined_df[col], downcast='integer')

combined_df.dtypes

Checking is the descriptive statistics of some of the variables changed after the data type conversion:

In [ ]:
combined_df[match_stats_columns_names].describe()

Deleting unnecessary columns:

In [ ]:
names_of_columns_to_drop = ["winner_seed", "winner_entry", "loser_seed", "loser_entry", "match_num"]

combined_df.drop(columns=names_of_columns_to_drop, inplace=True)

combined_df.columns

Creating new columns:

In [ ]:
# "total_points" - total points played in the match (sum of the serve points both the winner and the loser had).
combined_df["total_points"] = combined_df["w_svpt"] + combined_df["l_svpt"]

# "w_1st_percentage" - the winners' percentage of first serve in
combined_df["w_1st_percentage"] = combined_df["w_1stIn"]/combined_df["w_svpt"]
combined_df["w_1st_percentage"] = round(combined_df["w_1st_percentage"]*100, 3)

# "l_1st_percentage" - the losers' percentage of first serve in
combined_df["l_1st_percentage"] = combined_df["l_1stIn"]/combined_df["l_svpt"]
combined_df["l_1st_percentage"] = round(combined_df["l_1st_percentage"]*100, 3)

combined_df[["total_points", "w_svpt", "w_1stIn", "w_1st_percentage", "l_svpt", "l_1stIn", "l_1st_percentage"]].sample(10)

Getting the number of rows and columns in the dataset (post-cleaning):

In [ ]:
number_of_new_columns_added = 3 # Nothe that this doesn't automatically counts the numebr of new columns added to the DF. Update if necessary
number_of_rows_post_cleaning = combined_df.shape[0]
number_of_columns_post_cleaning = combined_df.shape[1] - number_of_new_columns_added

print(f"The dataset has {number_of_rows_post_cleaning} rows and {number_of_columns_post_cleaning+number_of_new_columns_added} columns.")
print(f"{number_of_rows_pre_cleaning-number_of_rows_post_cleaning} rows were deleted and {number_of_columns_pre_cleaning-number_of_columns_post_cleaning} columns were removed. {number_of_new_columns_added} new columns were created based on existing variabels.")

# Data Preprocessing

Creating a subset used in the multivariate analysis:

In [ ]:
col_names_multivariate_df = ["w_ace", "w_df", "w_svpt", "w_1stIn", "w_1stWon", "w_2ndWon", "w_SvGms", "w_bpSaved", "w_bpFaced", "l_ace", "l_df", "l_svpt", "l_1stIn", "l_1stWon", "l_2ndWon", "l_SvGms", "l_bpSaved", "l_bpFaced", "winner_ht", "winner_age", "loser_ht", "loser_age", "minutes", "total_points", "w_1st_percentage", "l_1st_percentage"]

multivariate_analysis_df = combined_df[col_names_multivariate_df]

multivariate_analysis_df.head()

Creating a new subset for the 1st serve by surface analysis:

In [ ]:
first_serve_by_surface_df = pd.melt(combined_df, id_vars = "surface", value_vars=["w_1st_percentage", "l_1st_percentage"])
first_serve_by_surface_df.sample(10)

# Univariate Analysis

## Categorical Variables

### Surface Type

Counting the number of games played on each surface:

In [ ]:
combined_df["surface"].value_counts()

Proportion of matches played on each surface:

In [ ]:
combined_df["surface"].value_counts(normalize=True)

Plotting the number of games played on each surface type:

In [ ]:
# Creating the plot
plt.figure(figsize=(12,6))
surfaces_countplot = sns.countplot(data=combined_df, x="surface", order=combined_df.surface.value_counts().index)

# Removing the frame
sns.despine(left=True, bottom=True)

# Adding Bar's Labels
surfaces_countplot.bar_label(surfaces_countplot.containers[0], label_type='edge')

# Titles
surfaces_countplot.set_xlabel("Type of Surface")
surfaces_countplot.set_ylabel("Number of Matches Played")
surfaces_countplot.set_title("Number of Matches Played Between 1991 and 2023* on each surface")

# Showing the plot
plt.show()

### Best of 5 VS Best of 3:

Counting the number of best of 5 and best of 3 matches played:

In [ ]:
combined_df["best_of"].value_counts()

Proportion of each best of- match:

In [ ]:
combined_df["best_of"].value_counts(normalize=True)

Plotting the number of best of 5 and best of 3 matches played:

In [ ]:
# Creating the plot
plt.figure(figsize=(12,6))
best_of_countplot = sns.countplot(data=combined_df, x="best_of", order=combined_df.best_of.value_counts().index)

# Removing the frame
sns.despine(left=True, bottom=True)

# Adding Bar's Labels
best_of_countplot.bar_label(best_of_countplot.containers[0], label_type='edge')

# Titles
best_of_countplot.set_xlabel("Best of How Many Sets")
best_of_countplot.set_ylabel("Number of Matches")
best_of_countplot.set_title("Number of Matches Played Between 1991 and 2023")

# Showing the plot
plt.show()

### Number of Matches by Tournament Type:

Counting the number of matches in each tournament type:

In [ ]:
combined_df["tourney_level"].value_counts()

Proportion of tournament type:

In [ ]:
combined_df["tourney_level"].value_counts(normalize=True)

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12,6))
tourney_level_countplot = sns.countplot(data=combined_df, x="tourney_level", order=combined_df["tourney_level"].value_counts().index)

# Removing the frame
sns.despine(left=True, bottom=True)

# Adding Bar's Labels
tourney_level_countplot.bar_label(tourney_level_countplot.containers[0], label_type='edge')

# Titles
tourney_level_countplot.set_xlabel("Tourney Type")
tourney_level_countplot.set_ylabel("Number of Matches")
tourney_level_countplot.set_title("Number of Matches Played Between 1991 and 2023 in each tourney type")

# Showing the plot
plt.show()

## Numerical Variables

### Analyzing the total_points variable:

Descriptive statistics:

In [ ]:
round(combined_df.total_points.describe(), 3)

Skewness and Kurtosis:

In [ ]:
total_points_skewness = combined_df.total_points.skew()
total_points_kurtosis = combined_df.total_points.kurtosis()

print(f"Skewness: {round(total_points_skewness, 3)}")
print(f"Kurtosis: {round(total_points_kurtosis, 3)}")

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12, 6))
total_point_histogram = sns.histplot(data=combined_df, x="total_points")

# Removing the frame
sns.despine(left=True, bottom=True)

# Titles
total_point_histogram.set_xlabel("Total Number of Points Played in a Match")
total_point_histogram.set_ylabel("Number of Matches")

# Showing the plot
plt.show()

### Analyzing the winner_age variable:

Descriptive statistics:

In [ ]:
round(combined_df["winner_age"].describe(), 3)

Skewness and Kurtosis:

In [ ]:
winner_age_skewness = combined_df["winner_age"].skew()
winner_age_kurtosis = combined_df["winner_age"].kurtosis()

print(f"Skewness: {round(winner_age_skewness, 3)}")
print(f"Kurtosis: {round(winner_age_kurtosis, 3)}")

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12, 6))
winner_age_histogram = sns.histplot(data=combined_df, x="winner_age")

# Removing the frame
sns.despine(left=True, bottom=True)

# Titles
winner_age_histogram.set_xlabel("Winners Age")
winner_age_histogram.set_ylabel("Number of Players")
winner_age_histogram.set_title("Winners Age Distribution")

# Showing the plot
plt.show()

### Analyzing the loser_age variable:

Descriptive statistics:

In [ ]:
combined_df["loser_age"].describe()

Skewness and Kurtosis:

In [ ]:
loser_age_skewness = combined_df["loser_age"].skew()
loser_age_kurtosis = combined_df["loser_age"].kurtosis()

print(f"Skewness: {round(loser_age_skewness, 3)}")
print(f"Kurtosis: {round(loser_age_kurtosis, 3)}")

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12, 6))
loser_age_histogram = sns.histplot(data=combined_df, x="loser_age")

# Removing the frame
sns.despine(left=True, bottom=True)

# Titles
loser_age_histogram.set_xlabel("Loser Age")
loser_age_histogram.set_ylabel("Number of Players")
loser_age_histogram.set_title("Losers Age Distribution")

# Showing the plot
plt.show()

### Analyzing the w_1st_percentage variable:

Descriptive statistics:

In [ ]:
combined_df["w_1st_percentage"].describe()

Skewness and Kurtosis:

In [ ]:
w_1st_percentage_skewness = combined_df["w_1st_percentage"].skew()
w_1st_percentage_kurtosis = combined_df["w_1st_percentage"].kurtosis()

print(f"Skewness: {round(w_1st_percentage_skewness, 3)}")
print(f"Kurtosis: {round(w_1st_percentage_kurtosis, 3)}")

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12, 6))
w_1st_percentage_histogram = sns.histplot(data=combined_df, x="w_1st_percentage")

# Removing the frame
sns.despine(left=True, bottom=True)

# Titles
w_1st_percentage_histogram.set_xlabel("Winner's First Serve In Percentage")
w_1st_percentage_histogram.set_ylabel("Number of Players")
w_1st_percentage_histogram.set_title("Winner's First Serve In Percentage Distribution")

# Showing the plot
plt.show()

### Analyzing the l_1st_percentage variable:

Descriptive statistics:

In [ ]:
combined_df["l_1st_percentage"].describe()

Skewness and Kurtosis:

In [ ]:
l_1st_percentage_skewness = combined_df["l_1st_percentage"].skew()
l_1st_percentage_kurtosis = combined_df["l_1st_percentage"].kurtosis()

print(f"Skewness: {round(l_1st_percentage_skewness, 3)}")
print(f"Kurtosis: {round(l_1st_percentage_kurtosis, 3)}")

Plotting:

In [ ]:
# Creating the plot
plt.figure(figsize=(12, 6))
l_1st_percentage_histogram = sns.histplot(data=combined_df, x="l_1st_percentage")

# Removing the frame
sns.despine(left=True, bottom=True)

# Titles
l_1st_percentage_histogram.set_xlabel("Loser's First Serve In Percentage")
l_1st_percentage_histogram.set_ylabel("Number of Players")
l_1st_percentage_histogram.set_title("Loser's First Serve In Percentage Distribution")

# Showing the plot
plt.show()

# Bivariate Analysis

## Finding trends in the numric variables:

* The goal of the pairplot visualization is to examine the relationships between all the numric variabels in the dataset. Though in many cases a correlation matrix is used to do so, I prefer to do this visualization in order to see if there's any non linear correlations between any variabels.

Creating a pairplot visualization:

In [ ]:
pairplot = sns.pairplot(multivariate_analysis_df, corner=True)
plt.show()

* I chose to create the correlation matrix in order to check the magnitude of the linear correlation between variabels.  

Creating a correlation matrix:

In [ ]:
corr_matrix = round(multivariate_analysis_df.corr(),2)

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

plt.figure(figsize=(20,12))
sns.heatmap(corr_matrix, annot=True, mask=mask, square=True, linewidths=2)
plt.show()

**Conclusions drawn from the pairplot visualization and correlation matrix:**  
* After examining the pairplot visualization, it seems that the only type of relationships are linear.
* From examining the correlation matrix, it seems that there is a strong correlation between some variabels. All of the strong correlations make sense - the correlation between the length of the match in minutes to the total number of points played in the match (*r=0.92*), for example, is easy to explain: if the match is longer, it means the players has more time to play more points.

## Comparing the average number of total points played on each surfce

Calculating each surface's average:

In [ ]:
total_points_mean_by_surface_df = round(combined_df.groupby(["surface"], as_index=False)[["total_points"]].mean(), 2)
total_points_mean_by_surface_df.sort_values(by="total_points", ascending=False, inplace=True, ignore_index=True)
total_points_mean_by_surface_df

Plotting:

In [ ]:
plt.figure(figsize=(16,9))
points_by_surface_plot = sns.boxplot(combined_df, x="surface", y="total_points", order=total_points_mean_by_surface_df["surface"])
sns.despine(left=True, bottom=True)
points_by_surface_plot.set_xlabel("Surface Type")
points_by_surface_plot.set_ylabel("Number of Points Played In A Match")
points_by_surface_plot.set_title("Total Points On Each Surface")
plt.show()

## Comparing the average match length for each event

In [ ]:
average_duration_by_tourney_level_df = round(combined_df.groupby(["tourney_level"], as_index=False)[["minutes", "total_points"]].mean(), 3)
average_duration_by_tourney_level_df.sort_values(by="minutes", ascending=False, inplace=True, ignore_index=True)
average_duration_by_tourney_level_df

In [ ]:
plt.figure(figsize=(16,9))
duration_by_tourney_level_plot = sns.boxplot(combined_df, x="tourney_level", y="minutes", order=average_duration_by_tourney_level_df["tourney_level"])
sns.despine(left=True, bottom=True)
duration_by_tourney_level_plot.set_xlabel("Tourney Level")
duration_by_tourney_level_plot.set_ylabel("Match Duration, in Minutes")
duration_by_tourney_level_plot.set_title("Match Duration in Minutes by Tourney Level")
plt.show()

## First Serve Percentage Difference Between Winners and Losers:

In [ ]:
round(combined_df[["w_1st_percentage", "l_1st_percentage"]].mean(), 3)

In [ ]:
plt.figure(figsize=(16,9))
sns.boxplot(data=combined_df[["w_1st_percentage", "l_1st_percentage"]])
sns.despine(left=True, bottom=True)

plt.show()

## Comparing the 1st serve percentages per surface

In [ ]:
first_serve_percentages_per_surface_df = round(first_serve_by_surface_df.groupby(["surface", "variable"], as_index=False).agg({"value": ["mean", "median"]}), 3)
first_serve_percentages_per_surface_df.sort_values(by=["surface", ("value", "median")], ascending=[True, False], inplace=True, ignore_index=True)
first_serve_percentages_per_surface_df

In [ ]:
plt.figure(figsize=(16,9))
sns.boxplot(data=first_serve_by_surface_df, x="surface", y="value", hue="variable")
sns.despine(left=True, bottom=True)